## Load text with tf.data

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.__version__)

### 搭建序贯模型

In [ ]:
model = tf.keras.Sequential([
    # 添加一个有 64 个神经元的全连接层，“input_shape”为该层接受的输入数据的维度
    # “activation”指定该层所用的激活函数
    layers.Dense(64, activation='relu', input_shape=(32,)),
    # 添加第二个网络层
    layers.Dense(64, activation='relu'),
    # 添加一个 softmax 层作为输出层，该层有十个单元
    layers.Dense(10, activation='softmax'),
    ])

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(32,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### 编译模型

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 使用numpy数据训练模型

In [ ]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
print(data[0])
print(labels[0])
model.fit(data, labels, epochs=2, batch_size=50)

### 使用验证集

In [ ]:
val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=2, batch_size=50,
          validation_data=(val_data, val_labels))

###  使用tf.data

In [ ]:
# 创建训练集Dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(50)
# 创建验证集Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(50)

model.fit(dataset, epochs=2, validation_data=val_dataset)

### 评估

In [ ]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

# 模型评估，测试集为NumPy数据
model.evaluate(data, labels, batch_size=50)
# 模型评估，测试集为Dataset数据
model.evaluate(dataset, steps=20)

### 预测

In [ ]:
result = model.predict(data, batch_size=50)
print(result[0])

## 构建高级模型

In [ ]:
# 单独的一个输入层
inputs = tf.keras.Input(shape=(32,))
# 网络层可以像函数一样被调用，其接收和输出的均为张量
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
# 输出层
predictions = layers.Dense(10, activation='softmax')(x)

In [ ]:
# 创建模型
model = tf.keras.Model(inputs=inputs, outputs=predictions)
# 编译模型
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 训练模型
model.fit(data, labels, epochs=2, batch_size=50)

# 自定义网络模型

In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    # 分类任务的类别数
    self.num_classes = num_classes
    # 定义我们自己的网络层
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 使用“__init__”方法中定义的网络层来构造网络的前馈过程
    x = self.dense_1(inputs)
    return self.dense_2(x)

In [ ]:
model = MyModel(num_classes=10)
# 编译模型
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 训练模型
model.fit(data, labels, batch_size=50, epochs=5)

In [ ]:
callbacks = [
  # 当验证集上的损失“val_loss”连续两个训练回合（epoch）都没有变化，则提前结束训练
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # 使用TensorBoard保存训练的记录，保存到“./logs”目录中
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=50, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

In [ ]:
# 创建一个简单的模型
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

# 将整个模型保存为HDF5文件
model.save('my_model')
# 加载保存的模型
model = tf.keras.models.load_model('my_model')

In [ ]:
# 将模型的权重参数保存为HDF5文件
model.save_weights('my_model.h5', save_format='h5')
# 重新加载
model.load_weights('my_model.h5')

# 将模型的结构保存为JSON文件
json_string = model.to_json()